# DEMO#3 - Re-run with randint and unique indices per random subset...

# DEMO#2 - Expand Experiments...

# DEMO#1 - 01/31/2024 - See https://jh-ml.nccs.nasa.gov/jupyterhub-prism/user/gtamkin/lab/workspaces/auto-0/tree/_AGB-dev/mpf-model-factories/MultiPathFusion/multi_path_fusion/notebooks/glenn/gt_mc_get_stats.ipynb

# Generate Permutation Importance (i.e., rank) from SHAP files

In [1]:
import shap
import numpy as np
import pickle
import glob
import pandas as pd
import subprocess
import csv
from pathlib import Path

In [2]:
def format_band_set(subset):
    indexListIntStr = ''
    count = 0
    for band in subset:
        if (count == 0):
            indexListIntStr = indexListIntStr + str(band)
        else:
            indexListIntStr = indexListIntStr + ', ' + str(band)
        count = count + 1
    return indexListIntStr


# Read from CSV

In [3]:
hyperspectral_aggregate_bandnames = '/explore/nobackup/projects/ilab/data/AGB/test/mcruns/input/bands/MLBS_2018_541567.6_4136443.0_542567.6_4137443.0-hyperspectral-aggregate-bandnames.csv'
only_row = None
with open(hyperspectral_aggregate_bandnames) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        only_row = row
        print(row[0])
        print(row[446])
#print(only_row)

ARI
2511.7


In [4]:
# initialize list of lists
headers = ['Group','Top10Index','Top10Rank','Top10Value','Scaling','Randomizer', '#Bands','#Trials','Band/Trial','MinBands','hh:mm:ss.ms','Host(s)','CPUs/Host','Concurrency','ResultDir','ID','Description','HF', 'Callbacks']
site = 'MLBS_2018_541567.6_4136443.0_542567.6_4137443.0'

## Read batch of shap files for experiment, which contains run-time artifacts (e.g., SHAP values)

In [5]:
root_data_path = '/explore/nobackup/projects/ilab/data/AGB/test/mcruns/output'

In [6]:
#headers = ['Group','Top10Index','Top10Rank','Scaling','Randomizer', '#Bands','#Trials','Band/Trial','MinBands','hh:mm:ss.ms','Host(s)','CPUs/Host','Concurrency','ResultDir','ID','Description','HF', 'Callbacks']
permutation_results_VI_3trials = [
     ['V1b1','NoTop10Index', 'Top10Rank', 'Top10Value','DivBy10K','randitU','21', '188', '3', '20','~0:27:00.00','gpu007','40','Pool', 
       'VI_randintU_20240302_188_20m_3b','575050868397492482', '## VI - 20 minimum occurrences, 3 bands per trial',
       'MLBS_2018_hyperspectral_indices.tif','EarlyWarning'],
     ['VIb1','NoTop10Index', 'Top10Rank', 'Top10Value','DivBy10K','randitU','21', '191', '3', '20','~0:27:00.00','gpu007','40','Pool', 
       'VI_randintU_20240302_191_20m_3b','972812867574749608', '## VI - 20 minimum occurrences, 3 bands per trial',
       'MLBS_2018_hyperspectral_indices.tif','EarlyWarning'],
     ['VIb1','NoTop10Index', 'Top10Rank', 'Top10Value','DivBy10K','randitU','21', '193', '3', '20','~0:27:00.00','gpu007','40','Pool', 
       'VI_randintU_20240302_193_20m_3b','853379430912402815', '## VI - 20 minimum occurrences, 3 bands per trial',
       'MLBS_2018_hyperspectral_indices.tif','EarlyWarning'],
     ['VIb1','NoTop10Index', 'Top10Rank', 'Top10Value','DivBy10K','randitU','21', '200', '3', '20','~0:27:00.00','gpu007','40','Pool', 
       'VI_randintU_20240302_200_20m_3b','875083503082867641', '## VI - 20 minimum occurrences, 3 bands per trial',
       'MLBS_2018_hyperspectral_indices.tif','EarlyWarning'],
     ['VIb1','NoTop10Index', 'Top10Rank', 'Top10Value','DivBy10K','randitU','21', '214', '3', '20','~0:27:00.00','gpu007','40','Pool', 
       'VI_randintU_20240302_214_20m_3b','279550132438151710', '## VI - 20 minimum occurrences, 3 bands per trial',
       'MLBS_2018_hyperspectral_indices.tif','EarlyWarning'],
     ['VIb1','NoTop10Index', 'Top10Rank', 'Top10Value','DivBy10K','randitU','21', '225', '3', '20','~0:27:00.00','gpu007','40','Pool', 
       'VI_randintU_20240302_225_20m_3b','424433684070503325', '## VI - 20 minimum occurrences, 3 bands per trial',
       'MLBS_2018_hyperspectral_indices.tif','EarlyWarning'],
     ['VIb1','NoTop10Index', 'Top10Rank', 'Top10Value','DivBy10K','randitU','21', '230', '3', '20','~0:27:00.00','gpu007','40','Pool', 
       'VI_randintU_20240302_230_20m_3b','500424548463509068', '## VI - 20 minimum occurrences, 3 bands per trial',
       'MLBS_2018_hyperspectral_indices.tif','EarlyWarning'],
     ['VIb1','NoTop10Index', 'Top10Rank', 'Top10Value','DivBy10K','randitU','21', '247', '3', '20','~0:27:00.00','gpu007','40','Pool', 
       'VI_randintU_20240302_247_20m_3b','498345501911623626', '## VI - 20 minimum occurrences, 3 bands per trial',
       'MLBS_2018_hyperspectral_indices.tif','EarlyWarning'],
#    
      ['VIb2','NoTop10Index', 'Top10Rank', 'Top10Value','DivBy10K','randitU','21', '188', '3', '20','~0:27:00.00','gpu007','40','Pool', 
       'VI/VI_randintU_20240302_188_20m_3b','575050868397492482', '## VI - 20 minimum occurrences, 3 bands per trial',
       'MLBS_2018_hyperspectral_indices.tif','EarlyWarning'],
     ['VIb2','NoTop10Index', 'Top10Rank', 'Top10Value','DivBy10K','randitU','21', '191', '3', '20','~0:27:00.00','gpu007','40','Pool', 
       'VI/VI_randintU_20240302_191_20m_3b','806398069851382119', '## VI - 20 minimum occurrences, 3 bands per trial',
       'MLBS_2018_hyperspectral_indices.tif','EarlyWarning'],
     ['VIb2','NoTop10Index', 'Top10Rank', 'Top10Value','DivBy10K','randitU','21', '193', '3', '20','~0:27:00.00','gpu007','40','Pool', 
       'VI/VI_randintU_20240302_193_20m_3b','874529206487521041', '## VI - 20 minimum occurrences, 3 bands per trial',
       'MLBS_2018_hyperspectral_indices.tif','EarlyWarning'],
     ['VIb2','NoTop10Index', 'Top10Rank', 'Top10Value','DivBy10K','randitU','21', '200', '3', '20','~0:27:00.00','gpu007','40','Pool', 
       'VI/VI_randintU_20240302_200_20m_3b','965060104529880310', '## VI - 20 minimum occurrences, 3 bands per trial',
       'MLBS_2018_hyperspectral_indices.tif','EarlyWarning'],
     ['VIb2','NoTop10Index', 'Top10Rank', 'Top10Value','DivBy10K','randitU','21', '214', '3', '20','~0:27:00.00','gpu007','40','Pool', 
       'VI/VI_randintU_20240302_214_20m_3b','728278796873995749', '## VI - 20 minimum occurrences, 3 bands per trial',
       'MLBS_2018_hyperspectral_indices.tif','EarlyWarning'],
     ['VIb2','NoTop10Index', 'Top10Rank', 'Top10Value','DivBy10K','randitU','21', '225', '3', '20','~0:27:00.00','gpu007','40','Pool', 
       'VI/VI_randintU_20240302_225_20m_3b','514050032010718833', '## VI - 20 minimum occurrences, 3 bands per trial',
       'MLBS_2018_hyperspectral_indices.tif','EarlyWarning'],
     ['VIb2','NoTop10Index', 'Top10Rank', 'Top10Value','DivBy10K','randitU','21', '230', '3', '20','~0:27:00.00','gpu007','40','Pool', 
       'VI/VI_randintU_20240302_230_20m_3b','360349938924147892', '## VI - 20 minimum occurrences, 3 bands per trial',
       'MLBS_2018_hyperspectral_indices.tif','EarlyWarning'],
     ['VIb2','NoTop10Index', 'Top10Rank', 'Top10Value','DivBy10K','randitU','21', '247', '3', '20','~0:27:00.00','gpu007','40','Pool', 
       'VI/VI_randintU_20240302_247_20m_3b','214107507827042739', 'VI - 20 minimum occurrences, 3 bands per trial',
       'MLBS_2018_hyperspectral_indices.tif','EarlyWarning'],
#    
      ['VIb3','NoTop10Index', 'Top10Rank', 'Top10Value','DivBy10K','randitU','21', '188', '3', '20','~0:27:00.00','gpu002','40','Pool', 
       'VI/VI_randintU_20240303_188_20m_3b','898188862101410804', '## VI - 20 minimum occurrences, 3 bands per trial',
       'MLBS_2018_hyperspectral_indices.tif','EarlyWarning'],
     ['VIb3','NoTop10Index', 'Top10Rank', 'Top10Value','DivBy10K','randitU','21', '191', '3', '20','~0:27:00.00','gpu002','40','Pool', 
       'VI/VI_randintU_20240303_191_20m_3b','101554869495136186', '## VI - 20 minimum occurrences, 3 bands per trial',
       'MLBS_2018_hyperspectral_indices.tif','EarlyWarning'],
     ['VIb3','NoTop10Index', 'Top10Rank', 'Top10Value','DivBy10K','randitU','21', '193', '3', '20','~0:27:00.00','gpu002','40','Pool', 
       'VI/VI_randintU_20240303_193_20m_3b','261914886327368313', '## VI - 20 minimum occurrences, 3 bands per trial',
       'MLBS_2018_hyperspectral_indices.tif','EarlyWarning'],
     ['VIb3','NoTop10Index', 'Top10Rank', 'Top10Value','DivBy10K','randitU','21', '200', '3', '20','~0:27:00.00','gpu002','40','Pool', 
       'VI/VI_randintU_20240303_200_20m_3b','552713534466588559', '## VI - 20 minimum occurrences, 3 bands per trial',
       'MLBS_2018_hyperspectral_indices.tif','EarlyWarning'],
     ['VIb3','NoTop10Index', 'Top10Rank', 'Top10Value','DivBy10K','randitU','21', '214', '3', '20','~0:27:00.00','gpu002','40','Pool', 
       'VI/VI_randintU_20240303_214_20m_3b','121679581342470880', '## VI - 20 minimum occurrences, 3 bands per trial',
       'MLBS_2018_hyperspectral_indices.tif','EarlyWarning'],
     ['VIb3','NoTop10Index', 'Top10Rank', 'Top10Value','DivBy10K','randitU','21', '225', '3', '20','~0:27:00.00','gpu002','40','Pool', 
       'VI/VI_randintU_20240303_225_20m_3b','664133247155684072', '## VI - 20 minimum occurrences, 3 bands per trial',
       'MLBS_2018_hyperspectral_indices.tif','EarlyWarning'],
     ['VIb3','NoTop10Index', 'Top10Rank', 'Top10Value','DivBy10K','randitU','21', '230', '3', '20','~0:27:00.00','gpu002','40','Pool', 
       'VI/VI_randintU_20240303_230_20m_3b','131603933973007654', '## VI - 20 minimum occurrences, 3 bands per trial',
       'MLBS_2018_hyperspectral_indices.tif','EarlyWarning'],
     ['VIb3','NoTop10Index', 'Top10Rank', 'Top10Value','DivBy10K','randitU','21', '247', '3', '20','~0:27:00.00','gpu007','40','Pool', 
       'VI/VI_randintU_20240303_247_20m_3b','120201914459639429', '## VI - 20 minimum occurrences, 3 bands per trial',
       'MLBS_2018_hyperspectral_indices.tif','EarlyWarning'],

]

In [7]:
#headers = ['Group','Top10Index','Top10Rank','Scaling','Randomizer', '#Bands','#Trials','Band/Trial','MinBands','hh:mm:ss.ms','Host(s)','CPUs/Host','Concurrency','ResultDir','ID','Description','HF', 'Callbacks']
permutation_results_VI = [
     ['VI','NoTop10Index', 'Top10Rank', 'Top10Value','DivBy10K','randitU','21', '32', '10', '10','0:27:25.246467','ilab213','30','Pool', 
       'VI_randint_U_20240229_6','891516017211874507', '## VI - ilab213:  Time elapsed on node [ilab213] = (hh:mm:ss.ms) 0:27:25.246467',
       'MLBS_2018_hyperspectral_indices.tif','EarlyWarning'],
     ['VI','NoTop10Index', 'Top10Rank', 'Top10Value','DivBy10K','randitU','21', '32', '10', '10','0:26:36.387707','ilab213','30','Pool', 
      'VI_randint_U_20240229_5','345536919353575236', '## VI - ilab213:  Time elapsed on node [ilab213] = (hh:mm:ss.ms) 0:26:36.387707',
      'MLBS_2018_hyperspectral_indices.tif','EarlyWarning'],
     ['VI','NoTop10Index', 'Top10Rank', 'Top10Value','DivBy10K','randitU','21', '32', '10', '10','0:25:12.317','ilab213','30','Pool', 
      'VI_randint_U_20240229_4','673843053471350852', '## VI - ilab213:  Time elapsed on node [ilab213] = (hh:mm:ss.ms) 0:25:12.317',
      'MLBS_2018_hyperspectral_indices.tif','EarlyWarning'],
     ['VI','NoTop10Index', 'Top10Rank', 'Top10Value','DivBy10K','randitU','21', '32', '10', '10','0:26:52.317282','ilab213','30','Pool', 
      'VI_randint_U_20240229_3','548764528868727045', '## VI - ilab213:  Time elapsed on node [ilab213] = (hh:mm:ss.ms) 0:26:52.317282',
      'MLBS_2018_hyperspectral_indices.tif','EarlyWarning'],
     ['VI','NoTop10Index', 'Top10Rank', 'Top10Value','DivBy10K','randitU','21', '32', '10', '10','0:26:56.519002','ilab213','30','Pool', 
      'VI_randint_U_20240229_2','418094143373180009', '## VI - ilab213:  Time elapsed on node [ilab213] = (hh:mm:ss.ms) 0:26:56.519002',
      'MLBS_2018_hyperspectral_indices.tif','EarlyWarning'],
     ['VI','NoTop10Index', 'Top10Rank', 'Top10Value','DivBy10K','randitU','21', '32', '10', '10','0:27:43.264823','ilab213','30','Pool', 
      'VI_randint_U_20240229_1','711241757083075094', '## VI - ilab213:  Time elapsed on node [ilab213] = (hh:mm:ss.ms) 0:27:43.264823',
      'MLBS_2018_hyperspectral_indices.tif','EarlyWarning'],
]

In [8]:
#headers = ['Group','Top10Index','Top10Rank','Top10Value','Scaling','Randomizer', '#Bands','#Trials','Band/Trial','MinBands','hh:mm:ss.ms','Host(s)','CPUs/Host','Concurrency','ResultDir','ID','Description','HF', 'Callbacks']
permutation_results_ALL_randint_U = [
     ['ALLu','NoTop10Index', 'Top10Rank', 'Top10Value', 'DivBy10K','randitU','447', '1016', '10', '10','??:13:25','gpu002','40','Pool', 
      'ALL_randintU_20240229_1016','983932495984268956', 
      'ALL - gpu002 - 10min - GOAL=1016 ~?? hours',
      'MLBS_2018_541567.6_4136443.0_542567.6_4137443.0-hyperspectral-aggregate.separate-background-scaled.tif','EarlyWarning'],    
     ['ALLu','NoTop10Index', 'Top10Rank', 'Top10Value', 'DivBy10K','randitU','447', '1032', '10', '10','??:13:25','gpu002','40','Pool', 
      'ALL_randintU_20240305_1032','763030221589651938', 
      'ALL - gpu002 - started 03/05/2024:  Min 10, Trials = 10, Batch = 40, Goal = 1032',
      'MLBS_2018_541567.6_4136443.0_542567.6_4137443.0-hyperspectral-aggregate.separate-background-scaled.tif','EarlyWarning'],    
     ['ALLu','NoTop10Index', 'Top10Rank', 'Top10Value', 'DivBy10K','randitU','447', '1653', '10', '20','??:13:25','gpu002','40','Pool', 
      'ALL_randintU_20240305_1653','882848942719320298', 
      'ALL - gpu002 - started 03/05/2024:  Min 20, Trials = 10, Batch = 40, Goal = 1653',
      'MLBS_2018_541567.6_4136443.0_542567.6_4137443.0-hyperspectral-aggregate.separate-background-scaled.tif','EarlyWarning'],    
     ['ALLu','NoTop10Index', 'Top10Rank', 'Top10Value', 'DivBy10K','randitU','447', '3108', '10', '50','??:13:25','gpu002','40','Pool', 
      'ALL_randintU_20240301_3108','195018083738905409', 
      'ALL - gpu002 - 50min - GOAL=3108',
      'MLBS_2018_541567.6_4136443.0_542567.6_4137443.0-hyperspectral-aggregate.separate-background-scaled.tif','EarlyWarning'],    
     ['ALLu','NoTop10Index', 'Top10Rank', 'Top10Value', 'DivBy10K','randitU','447', '6064', '10', '100','??:13:25','gpu002','40','Pool', 
      'ALL_randintU_20240301_6064','215041145914163553', 
      'ALL - ilab213 - 100m- GOAL=6064',
      'MLBS_2018_541567.6_4136443.0_542567.6_4137443.0-hyperspectral-aggregate.separate-background-scaled.tif','EarlyWarning'],    
]


In [9]:
#headers = ['Group','Top10Index','Top10Rank','Scaling','Randomizer', '#Bands','#Trials','Band/Trial','MinBands','hh:mm:ss.ms','Host(s)','CPUs/Host','Concurrency','ResultDir','ID','Description','HF', 'Callbacks']
permutation_results_NEON = [
     ['_NEON','NoTop10Index', 'Top10Rank', 'Top10Value', 'DivBy10K','randit','426', '5666', '10', '100', '07:00.00','gpu004','40','Pool', 
      'randint_sets_20240215_collection-NEON_max_occurrences-100_batch_size-10_num_samples-5666_bandLen-426','188577871478003488', 
      'NEON on gpu004 - min 10 occurrences: ~7hours',
      'MLBS_2018_Reflectance_reflectance_warp_scaled.tif','EarlyWarning'],
]


In [10]:
#permutation_results = permutation_results_ALL
#permutation_results = permutation_results_VI
#permutation_results = permutation_results_NEON
#permutation_results = permutation_results_ALL + permutation_results_NEON + permutation_results_VI
#permutation_results = permutation_results_ALL + permutation_results_NEON + permutation_results_VI
#permutation_results = permutation_results_VI_3trials
#permutation_results = permutation_results_ALL_randint_U + permutation_results_ALL  + permutation_results_NEON + permutation_results_VI_3trials + permutation_results_VI
permutation_results = permutation_results_ALL_randint_U  + permutation_results_NEON + permutation_results_VI_3trials + permutation_results_VI

print(len(permutation_results), '\n',permutation_results)

36 
 [['ALLu', 'NoTop10Index', 'Top10Rank', 'Top10Value', 'DivBy10K', 'randitU', '447', '1016', '10', '10', '??:13:25', 'gpu002', '40', 'Pool', 'ALL_randintU_20240229_1016', '983932495984268956', 'ALL - gpu002 - 10min - GOAL=1016 ~?? hours', 'MLBS_2018_541567.6_4136443.0_542567.6_4137443.0-hyperspectral-aggregate.separate-background-scaled.tif', 'EarlyWarning'], ['ALLu', 'NoTop10Index', 'Top10Rank', 'Top10Value', 'DivBy10K', 'randitU', '447', '1032', '10', '10', '??:13:25', 'gpu002', '40', 'Pool', 'ALL_randintU_20240305_1032', '763030221589651938', 'ALL - gpu002 - started 03/05/2024:  Min 10, Trials = 10, Batch = 40, Goal = 1032', 'MLBS_2018_541567.6_4136443.0_542567.6_4137443.0-hyperspectral-aggregate.separate-background-scaled.tif', 'EarlyWarning'], ['ALLu', 'NoTop10Index', 'Top10Rank', 'Top10Value', 'DivBy10K', 'randitU', '447', '1653', '10', '20', '??:13:25', 'gpu002', '40', 'Pool', 'ALL_randintU_20240305_1653', '882848942719320298', 'ALL - gpu002 - started 03/05/2024:  Min 20, Tri

In [11]:
# Create the pandas DataFrame
df = pd.DataFrame(permutation_results, columns=headers)
df

,Group,Top10Index,Top10Rank,Top10Value,Scaling,Randomizer,#Bands,#Trials,Band/Trial,MinBands,hh:mm:ss.ms,Host(s),CPUs/Host,Concurrency,ResultDir,ID,Description,HF,Callbacks
0,ALLu,NoTop10Index,Top10Rank,Top10Value,DivBy10K,randitU,447,1016,10,10,??:13:25,gpu002,40,Pool,ALL_randintU_20240229_1016,983932495984268956,ALL - gpu002 - 10min - GOAL=1016 ~?? hours,MLBS_2018_541567.6_4136443.0_542567.6_4137443....,EarlyWarning
1,ALLu,NoTop10Index,Top10Rank,Top10Value,DivBy10K,randitU,447,1032,10,10,??:13:25,gpu002,40,Pool,ALL_randintU_20240305_1032,763030221589651938,"ALL - gpu002 - started 03/05/2024: Min 10, Tr...",MLBS_2018_541567.6_4136443.0_542567.6_4137443....,EarlyWarning
2,ALLu,NoTop10Index,Top10Rank,Top10Value,DivBy10K,randitU,447,1653,10,20,??:13:25,gpu002,40,Pool,ALL_randintU_20240305_1653,882848942719320298,"ALL - gpu002 - started 03/05/2024: Min 20, Tr...",MLBS_2018_541567.6_4136443.0_542567.6_4137443....,EarlyWarning
3,ALLu,NoTop10Index,Top10Rank,Top10Value,DivBy10K,randitU,447,3108,10,50,??:13:25,gpu002,40,Pool,ALL_randintU_20240301_3108,195018083738905409,ALL - gpu002 - 50min - GOAL=3108,MLBS_2018_541567.6_4136443.0_542567.6_4137443....,EarlyWarning
4,ALLu,NoTop10Index,Top10Rank,Top10Value,DivBy10K,randitU,447,6064,10,100,??:13:25,gpu002,40,Pool,ALL_randintU_20240301_6064,215041145914163553,ALL - ilab213 - 100m- GOAL=6064,MLBS_2018_541567.6_4136443.0_542567.6_4137443....,EarlyWarning
5,_NEON,NoTop10Index,Top10Rank,Top10Value,DivBy10K,randit,426,5666,10,100,07:00.00,gpu004,40,Pool,randint_sets_20240215_collection-NEON_max_occu...,188577871478003488,NEON on gpu004 - min 10 occurrences: ~7hours,MLBS_2018_Reflectance_reflectance_warp_scaled.tif,EarlyWarning
6,V1b1,NoTop10Index,Top10Rank,Top10Value,DivBy10K,randitU,21,188,3,20,~0:27:00.00,gpu007,40,Pool,VI_randintU_20240302_188_20m_3b,575050868397492482,"## VI - 20 minimum occurrences, 3 bands per trial",MLBS_2018_hyperspectral_indices.tif,EarlyWarning
7,VIb1,NoTop10Index,Top10Rank,Top10Value,DivBy10K,randitU,21,191,3,20,~0:27:00.00,gpu007,40,Pool,VI_randintU_20240302_191_20m_3b,972812867574749608,"## VI - 20 minimum occurrences, 3 bands per trial",MLBS_2018_hyperspectral_indices.tif,EarlyWarning
8,VIb1,NoTop10Index,Top10Rank,Top10Value,DivBy10K,randitU,21,193,3,20,~0:27:00.00,gpu007,40,Pool,VI_randintU_20240302_193_20m_3b,853379430912402815,"## VI - 20 minimum occurrences, 3 bands per trial",MLBS_2018_hyperspectral_indices.tif,EarlyWarning
9,VIb1,NoTop10Index,Top10Rank,Top10Value,DivBy10K,randitU,21,200,3,20,~0:27:00.00,gpu007,40,Pool,VI_randintU_20240302_200_20m_3b,875083503082867641,"## VI - 20 minimum occurrences, 3 bands per trial",MLBS_2018_hyperspectral_indices.tif,EarlyWarning


# Loop through result files and create the ranking

In [12]:
#. Loop through result files and create the ranking
for r_index, row in df.iterrows():
    rank_file = '/explore/nobackup/projects/ilab/data/AGB/test/mcruns/output/rank/' + df.at[r_index,'ResultDir'] + '.rank' 
#    rank_file = '/explore/nobackup/projects/ilab/data/AGB/test/mcruns/output/rank/VI-20min-3trials/' + df.at[r_index,'ResultDir'] + '.rank' 
    my_file = Path(rank_file)
    if my_file.is_file():
        bandAbsSumArr = pickle.load(open(rank_file, "rb"))
        print('\n ======== index[', r_index, '] ==========')
        print('Loaded:', rank_file)
    else:
        shap_dir_prefix = root_data_path + '/' + df.at[r_index,'ResultDir'] + '/' + df.at[r_index,'ID'] 
        shap_dir = shap_dir_prefix + '/PERMUTATION_IMPORTANCE_VALUES/'
        print('\n ======== index[', r_index, '] ==========')
        print(r_index,'index: ', shap_dir)

        # Returns a list of names in list files. 
        shap_files = glob.glob(shap_dir + '**/*.shap_values0to50',  
                           recursive = True) 

        # set to max number of hyperspectral indices in experiment
        bandLen = int(df.at[r_index,'#Bands'])
        bandOccurenceArr = np.zeros(bandLen).astype(int)
        bandAbsSumArr = np.zeros(bandLen)

        first_time = True
        index = 0
        # Reload config
        start = left = '[['
        end = right = ']]'

        for shap_vals_path in shap_files:

            index = index+1
            num_shap_files = len(shap_files)
            bandListStr = shap_vals_path[shap_vals_path.index(left)+len(left):shap_vals_path.index(right)]
            bandList = [e for e in bandListStr.split(',')]
            for bandNum in bandList:
                bandOccurenceArr[int(bandNum)-1] = (bandOccurenceArr[int(bandNum)-1]) + 1

            # Reload shap values
            shap_values0to50 = pickle.load(open(shap_vals_path, "rb"))
            num_bins = len(shap_values0to50)
            num_rows = len(shap_values0to50[0])
            num_bands = num_cols = len(shap_values0to50[0][0])

            #  Walk across bins [0,4]
            for _bin in range(0, num_bins):

                # get absolute value of all values in matrix
                shap_values0to50_bin_abs = np.abs(shap_values0to50[_bin])

                # convert numpy matrix to dataframe and print sum of absolute values in each column
                df_shap_values0to50_bin_abs = pd.DataFrame(data=shap_values0to50_bin_abs)

                # create a row (numpy array) where each column is the sum of the absolute values in that vertical column
                shap_values_abs_sum_axis0 = np.sum(df_shap_values0to50_bin_abs, axis=0)

                # sort the row from low to high to determine the rank of the columns (e.g., features)
                shap_values_abs_sum_argsort = np.argsort(shap_values_abs_sum_axis0)

                # loop thru each column and update the corresponding statistics in the global row
                for _col in range(0, num_cols):
                    bandNum = int(bandList[_col])

                    # add the absolute value of the current shape_value to the existing value in the cell that is indexed by the band number
                    # this allows us to keep a running total shap values per feature for downselection later....
                    bandAbsSumArr[bandNum-1] = (bandAbsSumArr[bandNum-1] + df_shap_values0to50_bin_abs[_col].sum())

#        print('Summary array before sort:', bandAbsSumArr)
        ascending_indices = np.argsort(bandAbsSumArr)[:10]
        print('Top ten before after:', ascending_indices)
        minIndex = ascending_indices[0]

        descending_indices = np.argsort(bandAbsSumArr)[::-1][:10]
        maxIndex = descending_indices[0]

        pickle.dump(bandAbsSumArr, open(rank_file , "wb"))
        print('Saved:', rank_file)
        
    with np.printoptions(precision=5, suppress=True):
#        print('Final:', bandAbsSumArr[:10])
        print('Final.min:', bandAbsSumArr.min(), 'Final.max:', bandAbsSumArr.max())
        print('Final L->H:', np.argsort(bandAbsSumArr)[:10])
        row['Top10Index'] = np.argsort(bandAbsSumArr)[::-1][:10]

        indexList = np.argsort(bandAbsSumArr)[::-1][:10]
        bandList = []
        valueList = []
        for index in range(0, 10):
            bandList.append(str(only_row[indexList[index]]))
            valueList.append(str("{:3.3f}".format(bandAbsSumArr[indexList[index]])))
            print(indexList[index], only_row[indexList[index]],"{:3.3f}".format(bandAbsSumArr[indexList[index]]))        
            index = index + 1
        row['Top10Rank'] = bandList
        row['Top10Value'] = valueList

        print('Final Top10Rank Names:', bandList )
        print('Final Top10Rank Indices:', row['Top10Index'] )
        print('Final Top10Rank Cummulative Absolute SHAP values:', valueList)
        print('Final max:', row['Top10Index'][0])


 ======== index[ 0 ] ==========
Loaded: /explore/nobackup/projects/ilab/data/AGB/test/mcruns/output/rank/ALL_randintU_20240229_1016.rank
Final.min: 0.0005707408906238011 Final.max: 58.595956516838775
Final L->H: [ 1 25 26 27 28 38 33 35 40 44]
15 NIRv 58.596
7 LAI 7.278
8 MCTI 1.451
216 1359.7366 1.114
218 1369.7537 0.983
217 1364.7452 0.935
321 1885.633 0.802
327 1915.6842 0.800
320 1880.6244 0.682
220 1379.7708 0.671
Final Top10Rank Names: ['NIRv', 'LAI', 'MCTI', '1359.7366', '1369.7537', '1364.7452', '1885.633', '1915.6842', '1880.6244', '1379.7708']
Final Top10Rank Indices: [ 15   7   8 216 218 217 321 327 320 220]
Final Top10Rank Cummulative Absolute SHAP values: ['58.596', '7.278', '1.451', '1.114', '0.983', '0.935', '0.802', '0.800', '0.682', '0.671']
Final max: 15

 ======== index[ 1 ] ==========
Loaded: /explore/nobackup/projects/ilab/data/AGB/test/mcruns/output/rank/ALL_randintU_20240305_1032.rank
Final.min: 0.000539558646396946 Final.max: 36.73248678647368
Final L->H: [ 1 2

In [13]:
dfTest = df
dfTest

,Group,Top10Index,Top10Rank,Top10Value,Scaling,Randomizer,#Bands,#Trials,Band/Trial,MinBands,hh:mm:ss.ms,Host(s),CPUs/Host,Concurrency,ResultDir,ID,Description,HF,Callbacks
0,ALLu,"[15, 7, 8, 216, 218, 217, 321, 327, 320, 220]","[NIRv, LAI, MCTI, 1359.7366, 1369.7537, 1364.7...","[58.596, 7.278, 1.451, 1.114, 0.983, 0.935, 0....",DivBy10K,randitU,447,1016,10,10,??:13:25,gpu002,40,Pool,ALL_randintU_20240229_1016,983932495984268956,ALL - gpu002 - 10min - GOAL=1016 ~?? hours,MLBS_2018_541567.6_4136443.0_542567.6_4137443....,EarlyWarning
1,ALLu,"[15, 6, 8, 7, 217, 324, 21, 327, 320, 323]","[NIRv, fPAR, MCTI, LAI, 1364.7452, 1900.6586, ...","[36.732, 3.402, 1.591, 1.562, 1.044, 0.992, 0....",DivBy10K,randitU,447,1032,10,10,??:13:25,gpu002,40,Pool,ALL_randintU_20240305_1032,763030221589651938,"ALL - gpu002 - started 03/05/2024: Min 10, Tr...",MLBS_2018_541567.6_4136443.0_542567.6_4137443....,EarlyWarning
2,ALLu,"[15, 6, 7, 8, 217, 221, 218, 216, 327, 219]","[NIRv, fPAR, LAI, MCTI, 1364.7452, 1384.7793, ...","[78.415, 15.788, 9.907, 3.671, 1.820, 1.687, 1...",DivBy10K,randitU,447,1653,10,20,??:13:25,gpu002,40,Pool,ALL_randintU_20240305_1653,882848942719320298,"ALL - gpu002 - started 03/05/2024: Min 20, Tr...",MLBS_2018_541567.6_4136443.0_542567.6_4137443....,EarlyWarning
3,ALLu,"[15, 7, 6, 8, 218, 217, 219, 216, 324, 220]","[NIRv, LAI, fPAR, MCTI, 1369.7537, 1364.7452, ...","[99.323, 18.126, 13.150, 4.647, 3.748, 2.847, ...",DivBy10K,randitU,447,3108,10,50,??:13:25,gpu002,40,Pool,ALL_randintU_20240301_3108,195018083738905409,ALL - gpu002 - 50min - GOAL=3108,MLBS_2018_541567.6_4136443.0_542567.6_4137443....,EarlyWarning
4,ALLu,"[15, 7, 6, 8, 216, 217, 218, 219, 320, 221]","[NIRv, LAI, fPAR, MCTI, 1359.7366, 1364.7452, ...","[307.245, 57.360, 50.476, 11.075, 5.381, 5.245...",DivBy10K,randitU,447,6064,10,100,??:13:25,gpu002,40,Pool,ALL_randintU_20240301_6064,215041145914163553,ALL - ilab213 - 100m- GOAL=6064,MLBS_2018_541567.6_4136443.0_542567.6_4137443....,EarlyWarning
5,_NEON,"[305, 299, 303, 304, 199, 300, 198, 290, 301, ...","[1805.4965, 1775.4452, 1795.4792, 1800.4878, 1...","[5.156, 3.832, 3.658, 3.252, 3.024, 3.021, 2.6...",DivBy10K,randit,426,5666,10,100,07:00.00,gpu004,40,Pool,randint_sets_20240215_collection-NEON_max_occu...,188577871478003488,NEON on gpu004 - min 10 occurrences: ~7hours,MLBS_2018_Reflectance_reflectance_warp_scaled.tif,EarlyWarning
6,V1b1,"[15, 6, 7, 8, 4, 5, 13, 18, 10, 9]","[NIRv, fPAR, LAI, MCTI, EVI, EVI2, NDVI, SAVI,...","[45.762, 6.442, 4.465, 1.017, 0.278, 0.251, 0....",DivBy10K,randitU,21,188,3,20,~0:27:00.00,gpu007,40,Pool,VI_randintU_20240302_188_20m_3b,575050868397492482,"## VI - 20 minimum occurrences, 3 bands per trial",MLBS_2018_hyperspectral_indices.tif,EarlyWarning
7,VIb1,"[15, 7, 6, 8, 5, 4, 18, 13, 9, 10]","[NIRv, LAI, fPAR, MCTI, EVI2, EVI, SAVI, NDVI,...","[45.039, 10.946, 6.594, 1.353, 0.308, 0.270, 0...",DivBy10K,randitU,21,191,3,20,~0:27:00.00,gpu007,40,Pool,VI_randintU_20240302_191_20m_3b,972812867574749608,"## VI - 20 minimum occurrences, 3 bands per trial",MLBS_2018_hyperspectral_indices.tif,EarlyWarning
8,VIb1,"[15, 7, 6, 8, 5, 18, 13, 4, 9, 10]","[NIRv, LAI, fPAR, MCTI, EVI2, SAVI, NDVI, EVI,...","[57.775, 7.435, 3.428, 2.450, 0.266, 0.261, 0....",DivBy10K,randitU,21,193,3,20,~0:27:00.00,gpu007,40,Pool,VI_randintU_20240302_193_20m_3b,853379430912402815,"## VI - 20 minimum occurrences, 3 bands per trial",MLBS_2018_hyperspectral_indices.tif,EarlyWarning
9,VIb1,"[15, 6, 8, 7, 4, 5, 18, 13, 10, 9]","[NIRv, fPAR, MCTI, LAI, EVI, EVI2, SAVI, NDVI,...","[66.467, 3.432, 1.845, 1.524, 0.419, 0.347, 0....",DivBy10K,randitU,21,200,3,20,~0:27:00.00,gpu007,40,Pool,VI_randintU_20240302_200_20m_3b,875083503082867641,"## VI - 20 minimum occurrences, 3 bands per trial",MLBS_2018_hyperspectral_indices.tif,EarlyWarning


In [14]:
print('[ID]-[Group]-[Min]-[Band]-[Trials]-[Scaling]-[Top 10 Rank]-[Top 10 Indices]-[Randomizer]')
#headers = ['Group','Top10Index','Top10Rank','Scaling','Randomizer', '#Bands','#Trials','Band/Trial','MinBands','hh:mm:ss.ms','Host(s)','CPUs/Host','Concurrency','ResultDir','ID','Description','HF', 'Callbacks']
for index, row in df.iterrows():
    print('', str(index).rjust(2), '-',row['Group'].rjust(4),'-', row['MinBands'].rjust(3), '-', row['#Bands'].rjust(3),'-', row['#Trials'].rjust(4), '-', row['Scaling'].rjust(8), '-', row['Top10Rank'], '-', row['Top10Index'], '-', row['Randomizer'])

[ID]-[Group]-[Min]-[Band]-[Trials]-[Scaling]-[Top 10 Rank]-[Top 10 Indices]-[Randomizer]
  0 - ALLu -  10 - 447 - 1016 - DivBy10K - ['NIRv', 'LAI', 'MCTI', '1359.7366', '1369.7537', '1364.7452', '1885.633', '1915.6842', '1880.6244', '1379.7708'] - [ 15   7   8 216 218 217 321 327 320 220] - randitU
  1 - ALLu -  10 - 447 - 1032 - DivBy10K - ['NIRv', 'fPAR', 'MCTI', 'LAI', '1364.7452', '1900.6586', '383.072', '1915.6842', '1880.6244', '1895.65'] - [ 15   6   8   7 217 324  21 327 320 323] - randitU
  2 - ALLu -  20 - 447 - 1653 - DivBy10K - ['NIRv', 'fPAR', 'LAI', 'MCTI', '1364.7452', '1384.7793', '1369.7537', '1359.7366', '1915.6842', '1374.7622'] - [ 15   6   7   8 217 221 218 216 327 219] - randitU
  3 - ALLu -  50 - 447 - 3108 - DivBy10K - ['NIRv', 'LAI', 'fPAR', 'MCTI', '1369.7537', '1364.7452', '1374.7622', '1359.7366', '1900.6586', '1379.7708'] - [ 15   7   6   8 218 217 219 216 324 220] - randitU
  4 - ALLu - 100 - 447 - 6064 - DivBy10K - ['NIRv', 'LAI', 'fPAR', 'MCTI', '1359.73

In [15]:
pd.set_option('max_colwidth', 800)
pd.set_option('display.colheader_justify', 'left')
dfSummary = df[['Group','MinBands','Band/Trial','#Trials','#Bands','Randomizer','Scaling','Top10Rank','Top10Value']] 
dfSummary.style.set_properties(**{'text-align': 'left'})
dfSummary

,Group,MinBands,Band/Trial,#Trials,#Bands,Randomizer,Scaling,Top10Rank,Top10Value
0,ALLu,10,10,1016,447,randitU,DivBy10K,"[NIRv, LAI, MCTI, 1359.7366, 1369.7537, 1364.7452, 1885.633, 1915.6842, 1880.6244, 1379.7708]","[58.596, 7.278, 1.451, 1.114, 0.983, 0.935, 0.802, 0.800, 0.682, 0.671]"
1,ALLu,10,10,1032,447,randitU,DivBy10K,"[NIRv, fPAR, MCTI, LAI, 1364.7452, 1900.6586, 383.072, 1915.6842, 1880.6244, 1895.65]","[36.732, 3.402, 1.591, 1.562, 1.044, 0.992, 0.957, 0.823, 0.819, 0.744]"
2,ALLu,20,10,1653,447,randitU,DivBy10K,"[NIRv, fPAR, LAI, MCTI, 1364.7452, 1384.7793, 1369.7537, 1359.7366, 1915.6842, 1374.7622]","[78.415, 15.788, 9.907, 3.671, 1.820, 1.687, 1.489, 1.415, 1.321, 1.168]"
3,ALLu,50,10,3108,447,randitU,DivBy10K,"[NIRv, LAI, fPAR, MCTI, 1369.7537, 1364.7452, 1374.7622, 1359.7366, 1900.6586, 1379.7708]","[99.323, 18.126, 13.150, 4.647, 3.748, 2.847, 2.763, 2.647, 2.612, 2.381]"
4,ALLu,100,10,6064,447,randitU,DivBy10K,"[NIRv, LAI, fPAR, MCTI, 1359.7366, 1364.7452, 1369.7537, 1374.7622, 1880.6244, 1384.7793]","[307.245, 57.360, 50.476, 11.075, 5.381, 5.245, 4.933, 4.609, 4.592, 4.538]"
5,_NEON,100,10,5666,426,randit,DivBy10K,"[1805.4965, 1775.4452, 1795.4792, 1800.4878, 1274.5916, 1780.4537, 1269.583, 1730.3683, 1785.4622, 1790.4708]","[5.156, 3.832, 3.658, 3.252, 3.024, 3.021, 2.636, 2.605, 2.549, 2.506]"
6,V1b1,20,3,188,21,randitU,DivBy10K,"[NIRv, fPAR, LAI, MCTI, EVI, EVI2, NDVI, SAVI, NDII, MSI]","[45.762, 6.442, 4.465, 1.017, 0.278, 0.251, 0.245, 0.224, 0.173, 0.133]"
7,VIb1,20,3,191,21,randitU,DivBy10K,"[NIRv, LAI, fPAR, MCTI, EVI2, EVI, SAVI, NDVI, MSI, NDII]","[45.039, 10.946, 6.594, 1.353, 0.308, 0.270, 0.239, 0.186, 0.162, 0.119]"
8,VIb1,20,3,193,21,randitU,DivBy10K,"[NIRv, LAI, fPAR, MCTI, EVI2, SAVI, NDVI, EVI, MSI, NDII]","[57.775, 7.435, 3.428, 2.450, 0.266, 0.261, 0.232, 0.222, 0.147, 0.111]"
9,VIb1,20,3,200,21,randitU,DivBy10K,"[NIRv, fPAR, MCTI, LAI, EVI, EVI2, SAVI, NDVI, NDII, MSI]","[66.467, 3.432, 1.845, 1.524, 0.419, 0.347, 0.232, 0.196, 0.154, 0.130]"


In [16]:
from pandas import DataFrame
def left_align(df: DataFrame):
    left_aligned_df = df.style.set_properties(**{'text-align': 'left'})
    left_aligned_df = left_aligned_df.set_table_styles(
        [dict(selector='th', props=[('text-align', 'left')])]
    )
    return left_aligned_df


In [17]:
left_align(dfSummary)

,Group,MinBands,Band/Trial,#Trials,#Bands,Randomizer,Scaling,Top10Rank,Top10Value
0,ALLu,10,10,1016,447,randitU,DivBy10K,"['NIRv', 'LAI', 'MCTI', '1359.7366', '1369.7537', '1364.7452', '1885.633', '1915.6842', '1880.6244', '1379.7708']","['58.596', '7.278', '1.451', '1.114', '0.983', '0.935', '0.802', '0.800', '0.682', '0.671']"
1,ALLu,10,10,1032,447,randitU,DivBy10K,"['NIRv', 'fPAR', 'MCTI', 'LAI', '1364.7452', '1900.6586', '383.072', '1915.6842', '1880.6244', '1895.65']","['36.732', '3.402', '1.591', '1.562', '1.044', '0.992', '0.957', '0.823', '0.819', '0.744']"
2,ALLu,20,10,1653,447,randitU,DivBy10K,"['NIRv', 'fPAR', 'LAI', 'MCTI', '1364.7452', '1384.7793', '1369.7537', '1359.7366', '1915.6842', '1374.7622']","['78.415', '15.788', '9.907', '3.671', '1.820', '1.687', '1.489', '1.415', '1.321', '1.168']"
3,ALLu,50,10,3108,447,randitU,DivBy10K,"['NIRv', 'LAI', 'fPAR', 'MCTI', '1369.7537', '1364.7452', '1374.7622', '1359.7366', '1900.6586', '1379.7708']","['99.323', '18.126', '13.150', '4.647', '3.748', '2.847', '2.763', '2.647', '2.612', '2.381']"
4,ALLu,100,10,6064,447,randitU,DivBy10K,"['NIRv', 'LAI', 'fPAR', 'MCTI', '1359.7366', '1364.7452', '1369.7537', '1374.7622', '1880.6244', '1384.7793']","['307.245', '57.360', '50.476', '11.075', '5.381', '5.245', '4.933', '4.609', '4.592', '4.538']"
5,_NEON,100,10,5666,426,randit,DivBy10K,"['1805.4965', '1775.4452', '1795.4792', '1800.4878', '1274.5916', '1780.4537', '1269.583', '1730.3683', '1785.4622', '1790.4708']","['5.156', '3.832', '3.658', '3.252', '3.024', '3.021', '2.636', '2.605', '2.549', '2.506']"
6,V1b1,20,3,188,21,randitU,DivBy10K,"['NIRv', 'fPAR', 'LAI', 'MCTI', 'EVI', 'EVI2', 'NDVI', 'SAVI', 'NDII', 'MSI']","['45.762', '6.442', '4.465', '1.017', '0.278', '0.251', '0.245', '0.224', '0.173', '0.133']"
7,VIb1,20,3,191,21,randitU,DivBy10K,"['NIRv', 'LAI', 'fPAR', 'MCTI', 'EVI2', 'EVI', 'SAVI', 'NDVI', 'MSI', 'NDII']","['45.039', '10.946', '6.594', '1.353', '0.308', '0.270', '0.239', '0.186', '0.162', '0.119']"
8,VIb1,20,3,193,21,randitU,DivBy10K,"['NIRv', 'LAI', 'fPAR', 'MCTI', 'EVI2', 'SAVI', 'NDVI', 'EVI', 'MSI', 'NDII']","['57.775', '7.435', '3.428', '2.450', '0.266', '0.261', '0.232', '0.222', '0.147', '0.111']"
9,VIb1,20,3,200,21,randitU,DivBy10K,"['NIRv', 'fPAR', 'MCTI', 'LAI', 'EVI', 'EVI2', 'SAVI', 'NDVI', 'NDII', 'MSI']","['66.467', '3.432', '1.845', '1.524', '0.419', '0.347', '0.232', '0.196', '0.154', '0.130']"


## +++++++++

In [26]:
#random_set_path = '/explore/nobackup/projects/ilab/data/AGB/test/mcruns/input/random_sets/randint_sets_20240229_collection-ALL_min_occurrences-100_batch_size-10_num_samples-6064_bandLen-447.pkl'
#random_set_path = '/explore/nobackup/projects/ilab/data/AGB/test/mcruns/input/random_sets/randint_sets_20240229_collection-ALL_min_occurrences-50_batch_size-10_num_samples-3108_bandLen-447.pkl'
random_set_path = '/explore/nobackup/projects/ilab/data/AGB/test/mcruns/input/random_sets/randint_sets_20240301_collection-VI_min_occurrences-20_batch_size-3_num_samples-188_bandLen-21.pkl'
random_set_rows = pickle.load(open(random_set_path, "rb"))
num_bins = len(random_set_rows)
num_rows = len(random_set_rows[0])
num_bands = num_cols = len(random_set_rows[0][0])
print (random_set_path, num_bins, num_rows, num_bands,num_cols, '\n', random_set_rows[0])

/explore/nobackup/projects/ilab/data/AGB/test/mcruns/input/random_sets/randint_sets_20240301_collection-VI_min_occurrences-20_batch_size-3_num_samples-188_bandLen-21.pkl 188 3 1 1 
 ['1', '21', '14']


In [23]:
df_random_set_rows = pd.DataFrame(data=random_set_rows)
df_random_set_rows.head()
print(list(df_random_set_rows.columns))
df_random_set_rows_sorted0 = df_random_set_rows.sort_values(0, ascending=True)
df_random_set_rows_sorted1 = df_random_set_rows_sorted0.sort_values(1, ascending=True)
df_random_set_rows_sorted2 = df_random_set_rows_sorted1.sort_values(2, ascending=True)
df_random_set_rows_sorted123 = df_random_set_rows.sort_values(by = [0, 1,2 ], ascending = [True, True, True])

df_random_set_rows_sorted = df_random_set_rows_sorted123
#sorted_df = df_random_set_rows.sort_values(by = [0, 1], ascending = [True, True])
#print(sorted_df)
with pd.option_context('display.max_rows', None,):
#     print(sorted_df)
     print(df_random_set_rows_sorted.head())

[0, 1, 2]
     0   1   2
53   1  16  21
120  1  19  14
45   1   2   4
0    1  21  14
143  1   3   9


In [24]:
df_random_set_rows_sorted0

,0,1,2
0,1,21,14
53,1,16,21
110,1,5,16
36,1,6,7
143,1,3,9
...,...,...,...
59,9,19,12
121,9,21,20
103,9,13,1
1,9,21,7


In [25]:
# Find Duplicate rows
with pd.option_context('display.max_rows', None,):
    #print(df_random_set_rows.groupby(df_random_set_rows.columns.tolist(),as_index=False).size())
    print(df_random_set_rows.value_counts().head())

0   1   2 
2   15  18    2
19  4   18    2
4   1   2     1
3   12  16    1
        18    1
dtype: int64


In [17]:
import ast
bandNamePairList = list(ast.literal_eval("[['blue_ccdc', 'BAND-B']]"))
bandNamePairList

[['blue_ccdc', 'BAND-B']]

In [88]:
shap_vals_path = '/explore/nobackup/projects/ilab/data/AGB/test/mcruns/output/VI_randintU_20240302_188_20m_3b/575050868397492482/PERMUTATION_IMPORTANCE_VALUES/MLP_SGD_7_layers_1024_units_MAE_50_epochs_callbacks::575050868397492482.keras[[1, 2, 4]].shap_values0to50'
shap_values0to50 = pickle.load(open(shap_vals_path, "rb"))
num_bins = len(shap_values0to50)
num_rows = len(shap_values0to50[0])
num_bands = num_cols = len(shap_values0to50[0][0])
print (shap_vals_path, num_bins, num_rows, num_bands,num_cols, '\n', shap_values0to50[0])

/explore/nobackup/projects/ilab/data/AGB/test/mcruns/output/VI_randintU_20240302_188_20m_3b/575050868397492482/PERMUTATION_IMPORTANCE_VALUES/MLP_SGD_7_layers_1024_units_MAE_50_epochs_callbacks::575050868397492482.keras[[1, 2, 4]].shap_values0to50 5 50 3 3 
 [[ 7.69992669e-07  1.00036462e-07 -6.93957011e-06]
 [ 2.03003486e-06  2.69313653e-07 -1.47912900e-05]
 [ 3.70492538e-07  1.87953313e-07 -3.34973137e-06]
 [ 9.95248556e-07  9.44733620e-08 -6.71222806e-06]
 [-1.40815973e-06 -7.40587712e-08  1.21615827e-05]
 [-1.73866749e-06 -1.71363354e-07  1.46457553e-05]
 [ 3.16798687e-07 -2.35289335e-07 -2.60457397e-06]
 [-3.02096208e-07 -2.37425168e-08 -3.65575155e-08]
 [-1.80453062e-06 -8.52346420e-08  1.37016177e-05]
 [ 1.40925248e-06  3.56485446e-07 -1.28569702e-05]
 [ 9.58492359e-07  6.27835592e-08 -5.86892168e-06]
 [-5.79555829e-07  1.16477410e-07  1.18846695e-06]
 [-2.55852938e-07 -1.86562538e-07  1.45092607e-06]
 [ 2.02208757e-07 -5.06639482e-09  1.22860074e-06]
 [ 2.54710515e-07 -3.8395325

In [89]:
shap_values0to50_bin = shap_values0to50[_bin]
df_shap_values0to50_bin = pd.DataFrame(data=shap_values0to50_bin)
df_shap_values0to50_bin

,0,1,2
0,1.478195e-07,7.405877e-08,-7.034093e-06
1,3.995001e-07,1.856685e-07,-1.469895e-05
2,6.248554e-08,1.314779e-07,-3.474603e-06
3,1.880030e-07,6.685654e-08,-6.813755e-06
4,-2.714495e-07,-5.433957e-08,1.230394e-05
5,-3.313025e-07,-1.258155e-07,1.489396e-05
6,5.011757e-08,-1.690785e-07,-2.729545e-06
7,-6.169081e-08,-1.505017e-08,-1.640618e-07
8,-3.476938e-07,-6.382664e-08,1.390959e-05
9,2.709031e-07,2.452731e-07,-1.282692e-05


In [38]:
shap_values0to50_bin = shap_values0to50[_bin]
df_shap_values0to50_bin = pd.DataFrame(data=shap_values0to50_bin)
df_shap_values0to50_bin

,0,1,2,3,4,5,6,7,8,9
0,-0.000049,0.000000e+00,-0.000044,-0.000007,0.000000e+00,0.000000e+00,0.000000e+00,-0.000022,0.000000e+00,0.000009
1,-0.000086,0.000000e+00,-0.000086,-0.000011,-2.504587e-06,0.000000e+00,0.000000e+00,-0.000046,0.000000e+00,-0.000004
2,0.000022,0.000000e+00,0.000023,0.000004,0.000000e+00,0.000000e+00,0.000000e+00,0.000019,0.000000e+00,-0.000005
3,0.000011,0.000000e+00,0.000007,0.000004,0.000000e+00,-4.911234e-06,0.000000e+00,0.000012,0.000000e+00,0.000013
4,-0.000006,0.000000e+00,-0.000006,0.000005,-7.034228e-06,-1.888156e-06,-1.521310e-06,-0.000002,0.000000e+00,-0.000011
5,0.000071,0.000000e+00,0.000070,0.000004,4.264736e-06,1.968602e-06,0.000000e+00,0.000050,0.000000e+00,0.000016
6,0.000147,6.936972e-07,0.000152,0.000019,2.443971e-07,0.000000e+00,1.366446e-06,0.000086,1.000917e-06,0.000003
7,0.000072,0.000000e+00,0.000075,0.000006,4.969408e-06,1.756437e-06,2.879586e-07,0.000050,0.000000e+00,0.000020
8,-0.000213,0.000000e+00,-0.000218,-0.000021,-1.733255e-05,-5.582354e-07,-1.461408e-06,-0.000141,0.000000e+00,-0.000016
9,-0.000058,0.000000e+00,-0.000056,-0.000005,-3.361417e-06,0.000000e+00,0.000000e+00,-0.000028,0.000000e+00,-0.000003


In [39]:
shap_values0to50_bin_abs = np.abs(shap_values0to50[_bin])
df_shap_values0to50_bin_abs = pd.DataFrame(data=shap_values0to50_bin_abs)
df_shap_values0to50_bin_abs

,0,1,2,3,4,5,6,7,8,9
0,0.000049,0.000000e+00,0.000044,0.000007,0.000000e+00,0.000000e+00,0.000000e+00,0.000022,0.000000e+00,0.000009
1,0.000086,0.000000e+00,0.000086,0.000011,2.504587e-06,0.000000e+00,0.000000e+00,0.000046,0.000000e+00,0.000004
2,0.000022,0.000000e+00,0.000023,0.000004,0.000000e+00,0.000000e+00,0.000000e+00,0.000019,0.000000e+00,0.000005
3,0.000011,0.000000e+00,0.000007,0.000004,0.000000e+00,4.911234e-06,0.000000e+00,0.000012,0.000000e+00,0.000013
4,0.000006,0.000000e+00,0.000006,0.000005,7.034228e-06,1.888156e-06,1.521310e-06,0.000002,0.000000e+00,0.000011
5,0.000071,0.000000e+00,0.000070,0.000004,4.264736e-06,1.968602e-06,0.000000e+00,0.000050,0.000000e+00,0.000016
6,0.000147,6.936972e-07,0.000152,0.000019,2.443971e-07,0.000000e+00,1.366446e-06,0.000086,1.000917e-06,0.000003
7,0.000072,0.000000e+00,0.000075,0.000006,4.969408e-06,1.756437e-06,2.879586e-07,0.000050,0.000000e+00,0.000020
8,0.000213,0.000000e+00,0.000218,0.000021,1.733255e-05,5.582354e-07,1.461408e-06,0.000141,0.000000e+00,0.000016
9,0.000058,0.000000e+00,0.000056,0.000005,3.361417e-06,0.000000e+00,0.000000e+00,0.000028,0.000000e+00,0.000003


In [43]:
hap_values_sum_axis0 = np.sum(df_shap_values0to50_bin, axis=0)
hap_values_sum_axis0

0   -4.107953e-06
1   -7.435994e-06
2   -2.561900e-07
3    1.520580e-06
4    5.374354e-06
5   -2.851546e-06
6    3.952963e-06
7    1.135898e-05
8   -4.297067e-06
9   -3.228323e-06
dtype: float64

In [47]:
hap_values_sum_axis0_axis0 = np.sum(hap_values_sum_axis0, axis=0)
hap_values_sum_axis0_axis0

2.9802319056946618e-08

In [49]:
hap_values_sum_axis0_axis1 = np.sum(hap_values_sum_axis0, axis=1)
hap_values_sum_axis0_axis1

ValueError: No axis named 1 for object type Series

In [48]:
hap_values_sum = np.sum(df_shap_values0to50_bin)
hap_values_sum

0   -4.107953e-06
1   -7.435994e-06
2   -2.561900e-07
3    1.520580e-06
4    5.374354e-06
5   -2.851546e-06
6    3.952963e-06
7    1.135898e-05
8   -4.297067e-06
9   -3.228323e-06
dtype: float64

In [40]:
hap_values_abs_sum_axis0 = np.sum(df_shap_values0to50_bin_abs, axis=0)
hap_values_abs_sum_axis0

0    0.005515
1    0.000019
2    0.005538
3    0.000630
4    0.000223
5    0.000056
6    0.000025
7    0.003450
8    0.000013
9    0.001328
dtype: float64

In [42]:
hap_values_abs_sum = np.sum(df_shap_values0to50_bin_abs[0])
hap_values_abs_sum

0.005514640028161402

In [30]:
dfSort = dfTest.sort_values(["Group","ID"])
dfSort

,Group,Top10Index,Top10Rank,Scaling,Randomizer,#Bands,#Trials,Band/Trial,MinBands,hh:mm:ss.ms,Host(s),CPUs/Host,Concurrency,ResultDir,ID,Description,HF,Callbacks
0,VI,"[15, 6, 8, 7, 4, 5, 13, 10, 18, 9]","[NIRv, fPAR, MCTI, LAI, EVI, EVI2, NDVI, NDII, SAVI, MSI]",DivBy10K,randitU,21,32,10,10,0:26:36.387707,ilab213,30,Pool,VI_randint_U_20240229_5,345536919353575236,## VI - ilab213: Time elapsed on node [ilab213] = (hh:mm:ss.ms) 0:26:36.387707,MLBS_2018_hyperspectral_indices.tif,EarlyWarning
2,VI,"[15, 6, 7, 8, 4, 5, 13, 10, 18, 9]","[NIRv, fPAR, LAI, MCTI, EVI, EVI2, NDVI, NDII, SAVI, MSI]",DivBy10K,randitU,21,32,10,10,0:26:52.317282,ilab213,30,Pool,VI_randint_U_20240229_3,548764528868727045,## VI - ilab213: Time elapsed on node [ilab213] = (hh:mm:ss.ms) 0:26:52.317282,MLBS_2018_hyperspectral_indices.tif,EarlyWarning
1,VI,"[15, 7, 8, 4, 5, 13, 6, 10, 18, 9]","[NIRv, LAI, MCTI, EVI, EVI2, NDVI, fPAR, NDII, SAVI, MSI]",DivBy10K,randitU,21,32,10,10,0:25:12.317,ilab213,30,Pool,VI_randint_U_20240229_4,673843053471350852,## VI - ilab213: Time elapsed on node [ilab213] = (hh:mm:ss.ms) ??,MLBS_2018_hyperspectral_indices.tif,EarlyWarning
3,VI,"[20, 9, 1, 2, 3, 4, 5, 6, 7, 8]","[Albedo, MSI, CAI, CRI550, CRI700, EVI, EVI2, fPAR, LAI, MCTI]",DivBy10K,randitU,21,32,10,10,0:26:56.519002,ilab213,30,Pool,VI_randint_U_20240229_2,711241757083075094,## VI - ilab213: Time elapsed on node [ilab213] = (hh:mm:ss.ms) 0:26:56.519002,MLBS_2018_hyperspectral_indices.tif,EarlyWarning
4,VI,"[15, 6, 7, 8, 4, 5, 13, 18, 10, 9]","[NIRv, fPAR, LAI, MCTI, EVI, EVI2, NDVI, SAVI, NDII, MSI]",DivBy10K,randitU,21,32,10,10,0:27:43.264823,ilab213,30,Pool,VI_randint_U_20240229_1,711241757083075094,## VI - ilab213: Time elapsed on node [ilab213] = (hh:mm:ss.ms) 0:27:43.264823,MLBS_2018_hyperspectral_indices.tif,EarlyWarning
